# Olist E-commerce — Exploration & Funnel

This notebook explores the Olist Kaggle dataset and prepares inputs for the Sales Funnel Dashboard.

In [ ]:

import pandas as pd
from pathlib import Path

RAW = Path('data/raw')
display(RAW.resolve())


## Load CSVs

In [ ]:

import pandas as pd
import numpy as np

def read_csv(name):
    import glob, os
    matches = glob.glob(f"data/raw/{name}*.csv")
    assert matches, f"CSV for {name} not found in data/raw/"
    return pd.read_csv(matches[0])

orders = read_csv('olist_orders_dataset')
customers = read_csv('olist_customers_dataset')
payments = read_csv('olist_order_payments_dataset')
items = read_csv('olist_order_items_dataset')
products = read_csv('olist_products_dataset')

orders.head()


## Basic Cleaning & Derived Fields

In [ ]:

date_cols = ['order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date']
for c in date_cols:
    if c in orders.columns:
        orders[c] = pd.to_datetime(orders[c], errors='coerce')

def furthest_stage(row):
    stages = [
        ('created', pd.notna(row.get('order_purchase_timestamp'))),
        ('approved', pd.notna(row.get('order_approved_at'))),
        ('delivered_carrier', pd.notna(row.get('order_delivered_carrier_date'))),
        ('delivered_customer', pd.notna(row.get('order_delivered_customer_date'))),
    ]
    last = 'created'
    for name, ok in stages:
        if ok:
            last = name
    return last

orders['status_stage'] = orders.apply(furthest_stage, axis=1)
orders['order_purchase_month'] = orders['order_purchase_timestamp'].dt.to_period('M').dt.to_timestamp()
orders.head()


## Join Payments & Items

In [ ]:

payments_agg = payments.groupby('order_id').agg(total_paid=('payment_value','sum')).reset_index()
items['revenue'] = items['price'] + items['freight_value']
items_agg = items.groupby('order_id').agg(items=('order_item_id','count'), revenue=('revenue','sum')).reset_index()

master = (orders
          .merge(customers[['customer_id','customer_unique_id','customer_city','customer_state']], on='customer_id', how='left')
          .merge(payments_agg, on='order_id', how='left')
          .merge(items_agg, on='order_id', how='left'))
master.head()


## Save processed parquet

In [ ]:

import pathlib
OUT = pathlib.Path('data/processed'); OUT.mkdir(parents=True, exist_ok=True)
master.to_parquet(OUT / 'orders_master.parquet', index=False)
OUT.resolve()
